In [14]:
import glob, os
import pandas as pd

In [54]:
training_set = []
for label in glob.glob("./train/*"):
    for abs_path in glob.glob("./train/{0}/*".format(label.split('/')[-1])):
        training_set.append([abs_path, label.split('/')[-1]])
df_train = pd.DataFrame(training_set).rename(columns={0: "image", 1: "target"})

In [55]:
df_train['subject'] = df_train['image'].apply(lambda x: x.split('/')[-1].split('-')[1])
df_train['serial'] = df_train['image'].apply(lambda x: x.split('/')[-1].split('-')[2].split('.')[0])
df_train['image'] = df_train['image'].apply(lambda x: x.split('/')[-1])
df_train['source'] = 'train'
df_train

,image,target,subject,serial,source
0,NORMAL-4161487-6.jpeg,NORMAL,4161487,6,train
1,NORMAL-9103640-1.jpeg,NORMAL,9103640,1,train
2,NORMAL-9593905-3.jpeg,NORMAL,9593905,3,train
3,NORMAL-8115192-10.jpeg,NORMAL,8115192,10,train
4,NORMAL-1498414-1.jpeg,NORMAL,1498414,1,train
...,...,...,...,...,...
108304,DME-1896542-15.jpeg,DME,1896542,15,train
108305,DME-4445075-20.jpeg,DME,4445075,20,train
108306,DME-7658334-5.jpeg,DME,7658334,5,train
108307,DME-3757443-40.jpeg,DME,3757443,40,train


In [57]:
test_set = []
for label in glob.glob("./test/*"):
    for abs_path in glob.glob("./test/{0}/*".format(label.split('/')[-1])):
        test_set.append([abs_path, label.split('/')[-1]])
df_test = pd.DataFrame(test_set).rename(columns={0: "image", 1: "target"})

In [58]:
df_test['subject'] = df_test['image'].apply(lambda x: x.split('/')[-1].split('-')[1])
df_test['serial'] = df_test['image'].apply(lambda x: x.split('/')[-1].split('-')[2].split('.')[0])
df_test['image'] = df_test['image'].apply(lambda x: x.split('/')[-1])
df_test['source'] = 'test'
df_test

,image,target,subject,serial,source
0,NORMAL-7023622-2.jpeg,NORMAL,7023622,2,test
1,NORMAL-1990042-1.jpeg,NORMAL,1990042,1,test
2,NORMAL-5586445-1.jpeg,NORMAL,5586445,1,test
3,NORMAL-7920519-2.jpeg,NORMAL,7920519,2,test
4,NORMAL-1844016-1.jpeg,NORMAL,1844016,1,test
...,...,...,...,...,...
995,DME-943690-1.jpeg,DME,943690,1,test
996,DME-1805165-1.jpeg,DME,1805165,1,test
997,DME-943690-17.jpeg,DME,943690,17,test
998,DME-698993-3.jpeg,DME,698993,3,test


In [69]:
df = pd.concat([df_train, df_test])

In [71]:
df.to_csv('kermany_metadata.csv')

In [74]:
df.sample(frac=1)

,image,target,subject,serial,source
5235,NORMAL-1583447-1.jpeg,NORMAL,1583447,1,train
106639,DME-3064922-153.jpeg,DME,3064922,153,train
2765,NORMAL-3602939-3.jpeg,NORMAL,3602939,3,train
71461,CNV-7393104-44.jpeg,CNV,7393104,44,train
69299,CNV-1731375-8.jpeg,CNV,1731375,8,train
...,...,...,...,...,...
86842,CNV-904464-285.jpeg,CNV,904464,285,train
68338,CNV-1963682-82.jpeg,CNV,1963682,82,train
72157,CNV-2019423-1.jpeg,CNV,2019423,1,train
67090,CNV-172472-310.jpeg,CNV,172472,310,train
